## Reinforcement Learning Models

### Imports and Utils

In [90]:
"""
Importing necessary libraries
"""
import pygame
import neat
import os
import random
import numpy as np

import pickle

In [91]:
"""
Defining Utilities for the Game
"""
scale = 0.5
WIN_WIDTH = 1920 * scale
WIN_HEIGHT = 1080 * scale
CAR_WIDTH = 60 * scale
CAR_HEIGHT = 60 * scale

CAR_IMG = pygame.transform.scale(pygame.image.load(os.path.join("Images", "car.png")), (CAR_WIDTH, CAR_HEIGHT))
ROAD_IMGS = []
for i in range(1, 6):
    ROAD_IMGS.append(pygame.transform.scale(pygame.image.load(os.path.join("Images", f"map{i}.png")), (int(WIN_WIDTH), int(WIN_HEIGHT))))
BORDER_COLOR = (255, 255, 255, 255)

In [92]:
"""
Defining Car Class
"""
class Car:
    IMGS = CAR_IMG
    ROT_VEL = 5

    def __init__(self, position):
        self.position = position
        self.tilt = 0
        self.vel = 0

        self.image = self.IMGS
        self.center = (self.position[0] + self.image.get_width()//2, self.position[1] + self.image.get_height()//2)
        self.radars = []
        self.thetas = [0, 45, 90, 135, 180]

        self.dist = 0

    def engine(self, action):
        if action[0] == 1:
            self.vel += 2
            self.vel = min(self.vel, 10)
        if action[1] == 1:
            self.vel -= 2
            self.vel = max(self.vel, 0)
        if action[2] == 1:
            self.tilt += self.ROT_VEL
            self.tilt = self.tilt % 360
        if action[3] == 1:
            self.tilt -= self.ROT_VEL
            self.tilt = self.tilt % 360

    def move(self):
        self.dist += self.vel
        self.position[0] += self.vel*np.cos(np.radians(self.tilt))
        self.position[1] -= self.vel*np.sin(np.radians(self.tilt))
        self.center = (self.position[0] + self.image.get_width()//2, self.position[1] + self.image.get_height()//2)
    
    def draw(self, window):
        rotated_image = pygame.transform.rotate(self.image, self.tilt)
        new_rect = rotated_image.get_rect(center=self.image.get_rect(topleft=self.position).center)
        window.blit(rotated_image, new_rect.topleft)

    def get_radars(self, map):
        self.radars = []
        for i in self.thetas:
            length = 0
            x = self.center[0] + int(length*np.cos(np.radians(self.tilt + i)))
            y = self.center[1] + int(length*np.sin(np.radians(self.tilt + i)))
            while not map.get_at((x, y)) == BORDER_COLOR:
                length += 1
                x = self.center[0] + int(length*np.cos(np.radians(self.tilt + i)))
                y = self.center[1] + int(length*np.sin(np.radians(self.tilt + i)))
            self.radars.append((x, y), length)

    def get_mask(self):
        return pygame.mask.from_surface(self.image)
    


In [93]:
class Map:
    IMGS = ROAD_IMGS

    def __init__(self, road, position = [0, 0]):
        self.position = position
        self.image = self.IMGS[road]

        self.map_mask = self.get_mask()

    def collide(self, car):
        car_mask = car.get_mask()

        offset = (car.position[0] - self.position[0], car.position[1] - self.position[1])
        overlap = self.map_mask.overlap(car_mask, offset)

        return overlap is not None

    def get_mask(self):
        return pygame.mask.from_threshold(self.image, (255, 255, 255), (10, 10, 10, 255))     

In [94]:
"""
Game Structure and Functionality
"""
pygame.font.init()
RESTART_X = 830 * scale
RESTART_Y = 920 * scale
MAP_IND = 0

class RacingCarGame:

    def __init__(self, window, car, map):
        self.window = window
        self.car = car
        self.map = map

        self.clock = pygame.time.Clock()
        self.running = True
        self.score = 0

    def gameplay(self):
        game_over = False
        while self.running:
            self.clock.tick(30)
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    self.running = False
                if event.type == pygame.KEYDOWN:
                    if game_over and event.key == pygame.K_SPACE:
                        self.car = Car([RESTART_X, RESTART_Y])
                        self.map = Map(MAP_IND, [0, 0])
                        self.score = 0
                        game_over = False
                    if event.key == pygame.K_w:
                        self.car.engine([1, 0, 0, 0])
                    if event.key == pygame.K_s:
                        self.car.engine([0, 1, 0, 0])
                    if event.key == pygame.K_a:
                        self.car.engine([0, 0, 1, 0])
                    if event.key == pygame.K_d:
                        self.car.engine([0, 0, 0, 1])

            if not game_over:
                self.car.move()
                self.score = self.car.dist // 100
                if self.map.collide(self.car):
                    game_over = True

            if game_over:
                self.Game_Over()
            else:
                self.display()
        pygame.quit()

    def display(self):
        self.window.blit(self.map.image, (0, 0))
        self.car.draw(self.window)
        text = pygame.font.SysFont("norwester", 25).render(f"Score: {self.score}", 1, (0, 0, 0))
        self.window.blit(text, (10, 10))
        pygame.display.update()

    def Game_Over(self):
        self.window.blit(self.map.image, (0, 0))
        self.car.draw(self.window)
        text = pygame.font.SysFont("norwester", 25).render(f"Score: {self.score}", 1, (0, 0, 0))
        self.window.blit(text, (10, 10))
        text = pygame.font.SysFont("norwester", 50).render("Game Over", 1, (0, 0, 0))
        self.window.blit(text, (WIN_WIDTH//2 - text.get_width()//2, WIN_HEIGHT//2 - text.get_height()//2))
        pygame.display.update()

### Defining Game Structure

In [96]:
def main():
    car = Car([415, 460])
    map = Map(MAP_IND, [0, 0])
    window = pygame.display.set_mode((WIN_WIDTH, WIN_HEIGHT))

    pygame.font.init()
    game = RacingCarGame(window, car, map)
    game.gameplay()

main()

### Model Creation and Training

In [35]:
"""
Game Structure and Functionality
"""
pygame.font.init()
RESTART_X = 830 * scale
RESTART_Y = 920 * scale

class RacingCarGameAI:

    def __init__(self, window, car, map):
        self.window = window
        self.car = car
        self.map = map

        self.clock = pygame.time.Clock()
        self.running = True
        self.score = 0

    def gameplay(self):
        game_over = False
        while self.running:
            self.clock.tick(30)
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    self.running = False
                if event.type == pygame.KEYDOWN:
                    if game_over and event.key == pygame.K_SPACE:
                        self.car = Car([RESTART_X, RESTART_Y])
                        self.map = Map()
                        self.score = 0
                        game_over = False
                    if event.key == pygame.K_w:
                        self.car.engine([1, 0, 0, 0])
                    if event.key == pygame.K_s:
                        self.car.engine([0, 1, 0, 0])
                    if event.key == pygame.K_a:
                        self.car.engine([0, 0, 1, 0])
                    if event.key == pygame.K_d:
                        self.car.engine([0, 0, 0, 1])

            if not game_over:
                self.car.move()
                self.score = self.car.dist // 100
                if self.map.collide(self.car):
                    game_over = True

            if game_over:
                self.Game_Over()
            else:
                self.display()
        pygame.quit()

    def display(self):
        self.window.blit(ROAD_IMG, (0, 0))
        self.car.draw(self.window)
        text = pygame.font.SysFont("norwester", 25).render(f"Score: {self.score}", 1, (0, 0, 0))
        self.window.blit(text, (10, 10))
        pygame.display.update()

    def Game_Over(self):
        self.window.blit(ROAD_IMG, (0, 0))
        self.car.draw(self.window)
        text = pygame.font.SysFont("norwester", 25).render(f"Score: {self.score}", 1, (0, 0, 0))
        self.window.blit(text, (10, 10))
        text = pygame.font.SysFont("norwester", 50).render("Game Over", 1, (0, 0, 0))
        self.window.blit(text, (WIN_WIDTH//2 - text.get_width()//2, WIN_HEIGHT//2 - text.get_height()//2))
        pygame.display.update()

In [39]:
local_dir = os.getcwd()
config_path = os.path.join(local_dir, "Configs/config_RCAI.txt")
config = neat.config.Config(neat.DefaultGenome, neat.DefaultReproduction, neat.DefaultSpeciesSet, neat.DefaultStagnation, config_path)

population = neat.Population(config)
population.add_reporter(neat.StdOutReporter(True))
stats = neat.StatisticsReporter()
population.add_reporter(stats)
generation = 0

def main(genomes, config):
    global generation

    birds = []
    genes = []
    nets = []

    for _, g in genomes:
        net = neat.nn.FeedForwardNetwork.create(g, config)
        nets.append(net)
        birds.append(Bird(RESTART_X, RESTART_Y))
        g.fitness = 0
        genes.append(g)

    window = pygame.display.set_mode((WIN_WIDTH, WIN_HEIGHT))

    try:
        pygame.font.init()
        game = FlappyBirdGameAI(window, base, pipes, birds, genes, nets)
        game.gen = generation
        game.gameplay()
    except neat.CompleteGenome as e:
        raise e

    generation += 1

try:
    winner = population.run(main, 50)
except neat.CompleteGenome as e:
    winner = e.genome

with open("Models/FlappyBirdAI.pkl", "wb") as genome_file:
    pickle.dump(winner, genome_file)
print(f"The Best Genome:\n{winner}")


 ****** Running generation 0 ****** 

Population's average fitness: 3.45000 stdev: 9.19312
Best fitness: 43.50000 - size: (1, 3) - species 1 - id 16
Average adjusted fitness: 0.058
Mean genetic distance 1.685, standard deviation 0.582
Population of 20 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0    20     43.5    0.058     0
Total extinctions: 0
Generation time: 3.573 sec

 ****** Running generation 1 ****** 

Population's average fitness: 3.06000 stdev: 4.76187
Best fitness: 21.70000 - size: (1, 3) - species 1 - id 16
Average adjusted fitness: 0.095
Mean genetic distance 1.878, standard deviation 0.534
Population of 20 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    1    20     21.7    0.095     1
Total extinctions: 0
Generation time: 1.904 sec (2.739 average)

 ****** Running generation 2 ****** 

Population's average fitness: 92.97000 stdev: 1

### Genome Loading

In [40]:
with open("FlappyBirdAI.pkl", "rb") as genome_file:
    genome = pickle.load(genome_file)

local_dir = os.getcwd()
config_path = os.path.join(local_dir, "Configs/config_RCAI.txt")
config = neat.config.Config(neat.DefaultGenome, neat.DefaultReproduction, neat.DefaultSpeciesSet, neat.DefaultStagnation, config_path)

model = neat.nn.FeedForwardNetwork.create(genome, config)

In [45]:
print(model.activate((30, 100, 100)))

[-1.0]
